In [111]:
import pandas as pd
import numpy as np
import re
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', None)

# Chemin vers le fichier JSONL
file_path = '/home/carolus/Documents/school/green_ia/data/02_openfoodfacts_sample.jsonl'

# Lire le fichier JSONL en utilisant la méthode read_json de pandas avec l'option lines=True
df = pd.read_json(file_path, lines=True)

# Afficher le DataFrame
print(df)


    pnns_groups_1  \
0         unknown   
1         unknown   
2         unknown   
3         unknown   
4         unknown   
..            ...   
995       unknown   
996       unknown   
997       unknown   
998       unknown   
999       unknown   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  ingredients_tags  \
0                                                                                                    

In [112]:
# main preprocessing

df.rename(columns={'pnns_groups_1': 'groups'}, inplace=True)
df.rename(columns={'ingredients_tags': 'ingredients_temp'}, inplace=True)
df.rename(columns={'product_name': 'name'}, inplace=True)
df.rename(columns={'ecoscore_tags': 'ecoscore_groups'}, inplace=True)
df.rename(columns={'categories_tags': 'categories'}, inplace=True)
df.rename(columns={'ecoscore_score': 'ecoscore_note'}, inplace=True)
df.rename(columns={'labels_tags': 'labels'}, inplace=True)

# traitement col GROUPS 
df['groups'] = df['groups'].replace("unknown", None, regex=False)  

# traitement col NAME
df['name'] = df['name'].replace("", None)  
df['name'] = df['name'].replace({np.nan: None})

# traitement col INGREDIENTS
df['ingredients_temp'] = df['ingredients_temp'].replace("", None)  # remplace vide par None
df['ingredients_temp'] = df['ingredients_temp'].replace({np.nan: None}) # remplace NaN par None

df['ingredients_temp'] = df['ingredients_temp'].apply(lambda x: x if isinstance(x, list) else []) # remplace None par liste vide 
df['ingredients_temp'] = df['ingredients_temp'].apply(lambda x: ', '.join(x)) # converti liste en string 

# extraire éléments avec 'en:' nouvelle colonne
def extract_en_ingredients(ingredient_list):
    ingredients = ingredient_list.strip('[]').split(', ')
    return [ingredient.split(':')[-1] for ingredient in ingredients if ingredient.startswith('en:')]
df['ingredients'] = df['ingredients_temp'].apply(extract_en_ingredients)
df.drop(columns=['ingredients_temp'], inplace=True)
df['ingredients'] = df['ingredients'].apply(lambda x: ', '.join(x))
df['ingredients'] = df['ingredients'].replace("", None)  

# traitement col PACKAGING
df['packaging'] = df['packaging'].replace("", None)
def remove_two_letters_and_colon(s):
    if isinstance(s, str):
        return re.sub(r'\b\w{2}:\b', '', s)
    return s
df['packaging'] = df['packaging'].apply(remove_two_letters_and_colon)

# traitement col CODE
df['code'] = df['code'].apply(lambda x: np.nan if pd.isna(x) else int(round(x)))
df['code'] = df['code'].replace("", None)  
df['code'] = df['code'].replace({np.nan: None})

# supprime les lignes où le code ean ou le nom produit sont absents 
df = df[df['name'].notna() & df['code'].notna()]

In [113]:
df.tail(70)

,groups,packaging,name,ecoscore_groups,categories,ecoscore_note,labels,code,countries,ingredients
926,None,None,Caramel Chocolate Candies Sweet,[unknown],None,NaN,None,5051882153648,France,None
927,None,None,Caffè,[unknown],None,NaN,None,8002730125062,en:it,None
928,Composite foods,None,Mashed maple sweet potatoes,[unknown],[en:meals],NaN,None,36406304244,United States,"sweet-potato, vegetable, root-vegetable, tuber, water, maple-syrup, added-sugar, disaccharide, spice, condiment, sea-salt, salt, ginger"
929,Cereals and potatoes,"Paper insert, Plastic bag",Bread Quinoa Power Grains,[b],"[en:plant-based-foods-and-beverages, en:plant-based-foods, en:cereals-and-potatoes, en:specific-products, en:products-for-specific-diets, en:breads, en:products-without-gluten, en:gluten-free-breads]",72.0,"[en:no-gluten, en:organic, en:vegetarian, en:usda-organic, en:vegan, en:100-whole-grain, en:no-gums, en:no-starches]",866464000129,United States,"water, millet-flour, flour, cereal-flour, sorghum-flour, cereal, sorghum, quinoa-flour, plant, quinoa, buckwheat-flour, buckwheat, teff-flour, teff, psyllium, fiber, vegetable-fiber, sunflower-oil, oil-and-fat, vegetable-oil-and-fat, vegetable-oil, concentrated-grape-juice, fruit, berries, juice, fruit-juice, grape, grape-juice, flax-meal, salt, cultured-dextrose, black-sesame-seeds, brown-sesame-seeds"
930,None,None,Chickpea flour,[unknown],None,NaN,[en:no-gluten],9415748019626,en:au,None
931,None,None,Cioccolato di modica,[unknown],None,NaN,None,8056732681408,en:it,None
932,Milk and dairy products,None,Grana padano,[c],"[en:dairies, en:fermented-foods, en:fermented-milk-products, en:cheeses, en:italian-cheeses, en:grana-padano]",44.0,[en:pdo],8000430390094,Italia,None
933,None,None,Jay robb chocolate protein powder,[unknown],[en:dietary-supplements],NaN,"[en:no-added-sugar, en:no-lactose]",603907004104,en:us,None
934,None,Stück,Billa Tramezzini Ei,[unknown],None,NaN,None,9011465441896,"Frankreich, en:at",None
935,None,None,"Latta, Green Tea Aloe Vera",[unknown],None,NaN,None,610074987727,United States,"cane-sugar, added-sugar, disaccharide, sugar, japanese-bancha-green-tea, aloe-vera, vegetable, kombucha, microbial-culture, ferment, medusomyces-gisevi"


In [114]:
df['code'].head(10)

0     2488753035032
1     9310645338620
2      813622021809
3          17198431
4     4902560170994
5     3563210015730
6     4056489048855
7      810174011906
8      860000821131
10     886926216769
Name: code, dtype: object

In [115]:
nan_counts = df.isna().sum()

# Compter les listes vides
empty_list_counts = df.map(lambda x: x == []).sum()

# Afficher les résultats
print("nombre de NaN ou None par colonne :")
print(nan_counts)

nombre de NaN ou None par colonne :
groups             608
packaging          838
name                 0
ecoscore_groups      0
categories         494
ecoscore_note      705
labels             569
code                 0
countries            4
ingredients        695
dtype: int64


In [116]:
print("nombre de listes vides par colonne :")
print(empty_list_counts)

nombre de listes vides par colonne :
groups               0
packaging            0
name                 0
ecoscore_groups      0
categories          33
ecoscore_note        0
labels             105
code                 0
countries            0
ingredients          0
dtype: int64


In [117]:
df['labels'].head(50)

0                        [en:green-dot]
1                                  None
2                                  None
3                                  None
4                                  None
5                                  None
6                        [en:no-gluten]
7                                  None
8                                  None
10            [en:no-gluten, en:kosher]
12                                 None
13                                   []
14     [en:no-gmos, en:non-gmo-project]
15                  [en:no-added-sugar]
16                                 None
17                                 None
18      [en:no-gluten, en:no-additives]
19                                 None
20                       [en:green-dot]
21                                 None
22                                 None
23                             [en:pdo]
24                       [en:green-dot]
25     [en:no-gmos, en:non-gmo-project]
26                                 None


In [118]:
df['ingredients'].head(50)

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  None
1                                                                                                                                                                                                                                                                                                                                                                                                               